In [1]:
from myutils import common
%matplotlib inline
import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from cnocr import CnOcr
img_fp = './xm.jpg'
image1=cv2.imread(img_fp)
image=image1.copy()
height, width = image.shape[:2]

grayimage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
common.show(grayimage,"原图",cmap="gray",debug=False)
_,ostuImg=cv2.threshold(grayimage,0,255,cv2.THRESH_BINARY_INV+ cv2.THRESH_OTSU)
common.show(ostuImg,"原图",cmap="gray",debug=False)

img_dilated = cv2.dilate(ostuImg,  (400,400), iterations=25)
common.show(img_dilated,"膨胀图",cmap="gray",debug=True)

# 检测轮廓
contours, hierarchy = cv2.findContours(img_dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 获取最大的轮廓
largest_contour = max(contours, key=cv2.contourArea)
# 获取轮廓的外接矩形
rect = cv2.minAreaRect(largest_contour)
box = cv2.boxPoints(rect)
box = np.int0(box)
cv2.drawContours(image, [box], 0, (0, 0, 255), 2)
# common.show(image,"矩形框",cmap="gray",debug=True)

# 近似多边形
epsilon = 0.1 * cv2.arcLength(largest_contour, True)
approx = cv2.approxPolyDP(largest_contour, epsilon, True)
cv2.drawContours(image, [approx], -1, (0, 255, 0), 3)
common.show(image,"多边形",cmap="gray",debug=True)
n = []
#将三维数组转换为一维的xy数组
for x, y in zip(approx[:, 0, 0], approx[:, 0, 1]):
    n.append((x, y))
print(n)    
n = sorted(n)
print(n)   
sort_point = []
#获取x值最小的两个点，也就是最左边的两个点
n_point1 = n[:2]
#按y值进行排序，就知道哪个是左上角，哪个是左下角
n_point1.sort(key=lambda x: x[1])
sort_point.extend(n_point1)
#将x值比较大的两个点，也就是右边那两个点
n_point2 = n[2:4]
#按y值进行排序，就知道哪个是右上角，哪个是右下角，注意排序是从小大大
n_point2.sort(key=lambda x: x[1])
#reverse的话n_point2中y值大的排在前面
n_point2.reverse()
sort_point.extend(n_point2)
#最终p1数组中的点是左上，左下，右下，右上四个点
p1 = np.array(sort_point, dtype=np.float32)
print("sort_point",sort_point)
print(p1)
h = sort_point[1][1] - sort_point[0][1]
w = sort_point[2][0] - sort_point[1][0]
pts2 = np.array([[0, 0], [0, h], [w, h], [w, 0]], dtype=np.float32)

# 生成变换矩阵
M = cv2.getPerspectiveTransform(p1, pts2)
dst = cv2.warpPerspective(image1, M, (width, height))
common.show(dst,"变化图",cmap="gray",debug=True)
ocr = CnOcr(det_model_name='naive_det')  # 所有参数都使用默认值
# out = ocr.ocr(dst)
out = ocr.ocr_for_single_line(dst)
print(out)



[WARNING 2023-06-12 10:20:12,497 _showwarnmsg:110] D:\code\opencvlearn\myutils\common.py:58: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
 


[(94, 35), (19, 41), (25, 139), (102, 131)]
[(19, 41), (25, 139), (94, 35), (102, 131)]
sort_point [(19, 41), (25, 139), (102, 131), (94, 35)]
[[         19          41]
 [         25         139]
 [        102         131]
 [         94          35]]
{'text': '照文姆', 'score': 0.2974710166454315}
